In [18]:
import numpy as np
import PIL
from PIL import Image
import os

In [19]:
def load_resize(size,path):##"dataset/forest" to specific directory
    resize_pics=[] #<class 'PIL.Image.Image'> list
    for root,dirs,files in os.walk(path):
        for file in files:
            route=root+"/"+file

            #print(route)
            image=Image.open(route)
#             display(image)
            img = image.resize((size,size), PIL.Image.ANTIALIAS)
            
            resize_pics.append(img)
    return resize_pics #<class 'PIL.Image.Image'> list
    

In [20]:
resize_pics=load_resize(64,"dataset/forest")

In [21]:
def convert_to_gray(resize_pics):
    gray_pics=[]
    for pic in resize_pics:
        gray=pic.convert("L")
#         display(gray)
        gray_pics.append(gray)
    return gray_pics

In [22]:
gray_pics=convert_to_gray(resize_pics)

In [24]:
def rgb_pics_to_ndarray(resize_pics):
    rgb_pixs=[]
    for pic in resize_pics:
        pix=np.asarray(pic)
        rgb_pixs.append(pix)
    return rgb_pixs

In [25]:
rgb_pixs=rgb_pics_to_ndarray(resize_pics)

In [33]:
def gray_pics_to_ndarray(gray_pics):
    gray_pixs=[]
    for pic in gray_pics:
        pix=np.asarray(pic)
        gray_pixs.append(pix)
    return gray_pixs

In [35]:
gray_pixs=gray_pics_to_ndarray(gray_pics)

In [36]:
def load_pixs(size,path):
    resize_pics=load_resize(size,path)
    gray_pics=convert_to_gray(resize_pics)
    rgb_pixs=rgb_pics_to_ndarray(resize_pics)
    gray_pixs=gray_pics_to_ndarray(gray_pics)
    return rgb_pixs,gray_pixs
    
    

In [69]:
rgb,gray=load_pixs(64,"dataset/forest")

In [70]:
import matplotlib.pyplot as plt
def plot_rbg_pics(pixs):
    for pix in pixs:
        plt.imshow(pix)
        plt.show()
        
    

In [71]:
def nparray_to_image(pixs):
    pics=[]
    for pix in pixs:
        img = Image.fromarray(pix, 'RGB')
#         img = Image.fromarray(pix, 'L')  
#for black and white pics
        
        pics.append(img)
    return pics

In [72]:
def plot_gray_pics(pixs):
    for pix in pixs:
        plt.imshow(pix,cmap="gray")
        plt.show()
        
    

In [73]:

M = np.array([[0.412453, 0.357580, 0.180423],
              [0.212671, 0.715160, 0.072169],
              [0.019334, 0.119193, 0.950227]])
def __rgb_to_xyz__(rgb_pixel):
    xyz_pixel = np.dot(M, rgb_pixel.T)
    xyz_pixel = xyz_pixel/255.0
    xyz_pixel = xyz_pixel[0] / 0.95047, xyz_pixel[1] / 1.0, xyz_pixel[2] / 1.08883
    return xyz_pixel

def f(im_channel):
    return np.power(im_channel, 1 / 3) if im_channel > 0.008856 else 7.787 * im_channel + 0.137931

def __xyz_to_lab__(xyz_pixel):
    F_XYZ = [f(x) for x in xyz_pixel]
    L = 116 * F_XYZ[1] - 16 if xyz_pixel[1] > 0.008856 else 903.3 * xyz_pixel[1]
    a = 500 * (F_XYZ[0] - F_XYZ[1])
    b = 200 * (F_XYZ[1] - F_XYZ[2])
    return (L, a, b)

def rgb_to_lab(rgb_pixs, size):
    lab_pixs=[]
    for rgb_pix in rgb_pixs:
        lab_pix = []
        for y in range(size[1]):
            row=[]
            for x in range(size[0]):
                xyz = __rgb_to_xyz__(np.array(rgb_pix[y][x]))
                lab = __xyz_to_lab__(xyz)
                row.append(lab)
            lab_pix.append(row)
        lab_pixs.append(lab_pix)
    return lab_pixs


In [78]:
lab_pixs=rgb_to_lab(rgb,(64,64))

In [84]:
def get_lab_channels(lab_pixs, size):
    lab_channel_pixs=[]
    for lab_pix in lab_pixs:
        channel_l=[]
        channel_a=[]
        channel_b=[]
        for y in range(size[1]):
            row_l=[]
            row_a=[]
            row_b=[]
            for x in range(size[0]):
                row_l.append(lab_pix[y][x][0])
                row_a.append(lab_pix[y][x][1])
                row_b.append(lab_pix[y][x][2])
            channel_l.append(row_l)
            channel_a.append(row_a)
            channel_b.append(row_b)

        lab_channel_pixs.append(np.array([channel_l, channel_a, channel_b]))
    return lab_channel_pixs


In [85]:
lab_channels_pixs = get_lab_channels(lab_pixs, (64,64))

In [86]:
def get_features(lab_channels_pixs):
    channel_l = []
    channel_ab = []
    for lab_channels_pix in lab_channels_pixs:
        channel_l.append(np.array(lab_channels_pix[0]).reshape(1,-1))
        channel_ab.append(np.append(np.array(lab_channels_pix[1]).reshape(1,-1), np.array(lab_channels_pix[2]).reshape(1,-1)))
    return np.array(channel_l).reshape(-1,size[0]*size[1]), np.add(np.array(channel_ab), 128)

In [87]:
size=(64,64)

In [89]:
features, labels = get_features(lab_channels_pixs)  # L [0,100] A,B[0,255] #feature[array(4096),...] labels[array(8192)]

In [111]:
train_test_split = np.random.rand(17) < 0.70

In [108]:
train_x = features[train_test_split]
train_y = labels[train_test_split]

In [117]:
test_x = features[~train_test_split]
test_y = labels[~train_test_split]

In [118]:
import tensorflow as tf

(17, 8192)
